# Pourbaix diagram from experimental values

To calculate a Pourbaix diagram, we need to calculate the energy of the possible phases for different electrode potentials and pH. The phase with the lowest energy is the most stable.


## Chemical potential
But what do we mean exactly by 'energy'? Let's say the total internal energy of our system is $U$. In electrochemistry, we usually have a fixed temperature $T$ and pressure $p$. We want to exclude the heat and volume work contributions from the environment, so we use the **Gibbs free energy**

$$
G = U - TS + pV
$$

with $S$ the entropy and $V$ the volume of the system. You can think of the Gibbs free energy as the 'chemical energy' in the system.

Equivalently, we can use the chemical potential $\mu = G/N$, which is just the Gibbs energy per particle. So: **we want to find the phase with the lowest chemical potential at different electrode potentials and pH**.

## Formation energy

In general, the chemical potential of a species $i$ is

$$
\mu_i = \mu_i^0 + k_\mathrm{B} T \ln a_i
$$

where $\mu_i^0$ is the chemical potential at standard conditions and $a_i$ is the activity. For dissolved species, the activity is the same as the concentration in M (assuming relatively low concentrations, which we always have in this case). For solids, $a_i=1$.

The standard chemical potential is the same as the standard Gibbs free energy of formation per particle:

$$
\mu_i^0 = \Delta_\mathrm{f}G_i^0.
$$

The formation energy of an element in its most stable state at standard conditions (gas for oxygen and hydrogen, solid for metals, etc.) is zero. (The elements are taken as a reference point!) For other species, values can be found online, like in the supplementary material of {cite:t}`wang2020predicting`.

Let's get calculating! Collect tabulated values for your solid of interest (and remember that you need to cite the source in your report). The experimental value of water can be found in {cite:t}`persson2012prediction` (section C).

In [ ]:
standard_chemical_potentials = {
    "Fe": 0,
    "FeO": -2.553,
    "Fe3O4": -10.524,
    "Fe2O3": -7.693,
    "Fe2+": -0.817,
    "Fe3+": -0.048,
    "H2O": -2.46,
}

Now, we calculate the reaction energy of forming each phase from the solid metal phase. A metal oxide, for example, is formed as

$$

$$


```{bibliography}
:style: unsrt
:filter: docname in docnames
```